In [2]:
import numpy as np
from catboost import CatBoostRegressor, Pool
import datetime
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
import optuna
import pandas as pd
from optuna.samplers import TPESampler
import seaborn as sns
from sklearn.compose import ColumnTransformer
from sklearn.decomposition import PCA
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.preprocessing import OneHotEncoder , OrdinalEncoder, StandardScaler

In [3]:
def dataset_cleaning(dataset):
    df = pd.read_csv(dataset)
    df2 = df.copy().dropna(subset=["Rating","Content Rating", "Current Ver", "Android Ver"])
    df2["Reviews"] = df2["Reviews"].astype('int64')
    df2["Size"] = df2["Size"].replace(['Varies with device'],['14000']).apply(lambda x: float(x.replace('M','')) *1000 if 'M' in x else (float(x.replace('k','')) /1000 if 'k' in x else x)).astype('float64')
    df2["Installs"] = df2["Installs"].str.split('+',expand=True)[0].apply(lambda x: x.replace(',','')).astype('int64')
    df2["Main_Genre"] = df2["Genres"].str.split(';',expand=True)[0]
    df2["Last Updated"] = df2["Last Updated"].apply(lambda x: x.replace(' ','/').replace(',','').replace('January','1').replace('February','2').replace('March','3').replace('April','4').replace('May','5').replace('June','6').replace('July','7').replace('August','8').replace('September','9').replace('October','10').replace('November','11').replace('December','12')).astype('datetime64')
    timestamp = pd.Timestamp(datetime.datetime(2021, 10, 10))
    df2['Today'] = pd.Timestamp(timestamp.today().strftime('%d-%m-%Y'))
    df2['Days_Since_Last_Update'] = (df2['Today'] - df2["Last Updated"]).dt.days
    df2.drop(['App','Type','Price','Genres','Today','Last Updated'], axis=1, inplace=True)
    df3 = df2.copy()
    return df3

In [10]:
df_clean = dataset_cleaning(r'..\data\raw_files\googleplaystore.csv')

In [11]:
df_clean.shape

(9360, 10)

In [12]:
df_clean.corr()

C:\Users\Victor\AppData\Local\Temp\ipykernel_8156\1863484124.py:1: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  df_clean.corr()


,Rating,Reviews,Size,Installs,Days_Since_Last_Update
Rating,1.000000,0.068133,0.067508,0.051337,-0.142966
Reviews,0.068133,1.000000,0.103867,0.641605,-0.088182
Size,0.067508,0.103867,1.000000,0.044622,-0.193577
Installs,0.051337,0.641605,0.044622,1.000000,-0.104371
Days_Since_Last_Update,-0.142966,-0.088182,-0.193577,-0.104371,1.000000


In [12]:
df_clean.to_csv(r'..\data\processed_data\test_saving_Data.csv', index = False)

In [ ]:
paths = {"raw_data":"\data\raw_files\","preprocessed_data":"\data\preprocessed_files\","\model\"]

In [23]:
paths = [r"../data/raw_data/",r"../data/preprocessed_data/",r"../model/"]


In [24]:
paths[0]

'../data/raw_data/'